In [2]:

import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import pyodbc  as sapsql 
 
datacompany = (
                {'host':'192.168.250.40' , 'dbname':'IGU_NEW','user':'sa','password':'B1admin'}, 
                )
                
datalist=[]
listcom = []
for company in datacompany:
    
    msg_sql=  """
                    declare @dateto varchar(20)


                    set @dateto = convert(varchar,getdate(),112)
                    --select  * from ocrg  where grouptype='C'


                    select  'proyeksi' Header,
                            right(convert(varchar,a.DueDate ,112),2) iday, 
                            case 
                            when d.GroupCode in (100,102,120) then '01-CATERING'
                            when d.GroupCode in (103) then '02-HOREKA'
                            when d.GroupCode in (105) then '03-RETAIL'
                            when d.GroupCode in (106) then '04-PASTRY'
                            when d.GroupCode in (107,120) then '05-QSR'
                            when d.GroupCode in (109,110) then '06-WET'
                            when d.GroupCode in (114) then '07-ECOMMERCE'
                            when d.GroupCode in (108,116,117,118,121) then '08-GROUP'
                            else   '09-OTHERS'
                            end idivisi,
                            sum(a.debit-a.credit) amount
                    from jdt1 A
                    inner join ojdt B ON A.transid = b.transid 
                    inner join ocrd c on a.ShortName = c.cardcode 
                    inner join ocrg d on c.groupcode = d.groupcode 

                    where left(convert(varchar,a.DueDate ,112),6) = left(@dateto,6)
                    and a.account ='1130001' and a.transtype in (13,14)
                    group by right(convert(varchar,a.DueDate ,112),2) ,
                            case 
                            when d.GroupCode in (100,102,120) then '01-CATERING'
                            when d.GroupCode in (103) then '02-HOREKA'
                            when d.GroupCode in (105) then '03-RETAIL'
                            when d.GroupCode in (106) then '04-PASTRY'
                            when d.GroupCode in (107,120) then '05-QSR'
                            when d.GroupCode in (109,110) then '06-WET'
                            when d.GroupCode in (114) then '07-ECOMMERCE'
                            when d.GroupCode in (108,116,117,118,121) then '08-GROUP'
                            else   '09-OTHERS'
                            end

                    union all 

                    select  'realisasi' Header,
                            right(convert(varchar,a.RefDate ,112),2) iday,
                            case 
                                when d.GroupCode in (100,102,120) then '01-CATERING'
                                when d.GroupCode in (103) then '02-HOREKA'
                                when d.GroupCode in (105) then '03-RETAIL'
                                when d.GroupCode in (106) then '04-PASTRY'
                                when d.GroupCode in (107,120) then '05-QSR'
                                when d.GroupCode in (109,110) then '06-WET'
                                when d.GroupCode in (114) then '07-ECOMMERCE'
                                when d.GroupCode in (108,116,117,118,121) then '08-GROUP'
                                else   '09-OTHERS'
                            end,
                            
                            sum(a.debit-a.credit)
                    from jdt1 A
                    inner join ojdt B ON A.transid = b.transid 
                    inner join ocrd c on a.ShortName = c.cardcode 
                    inner join ocrg d on c.groupcode = d.groupcode 

                    where left(convert(varchar,a.refdate ,112),6) = left(@dateto,6)
                    and a.account ='1130001' and a.transtype in (24) and left(b.U_Trans_No,2) in ('BD','KD')
                    group by right(convert(varchar,a.refdate ,112),2) ,
                            case 
                            when d.GroupCode in (100,102,120) then '01-CATERING'
                            when d.GroupCode in (103) then '02-HOREKA'
                            when d.GroupCode in (105) then '03-RETAIL'
                            when d.GroupCode in (106) then '04-PASTRY'
                            when d.GroupCode in (107,120) then '05-QSR'
                            when d.GroupCode in (109,110) then '06-WET'
                            when d.GroupCode in (114) then '07-ECOMMERCE'
                            when d.GroupCode in (108,116,117,118,121) then '08-GROUP'
                            else   '09-OTHERS'
                            end

    """
     
    listcom.append(company["dbname"])
    conn = sapsql.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+ company["host"] +';DATABASE='+ company["dbname"] +';UID='+ company["user"] +';PWD='+ company["password"] + ';TrustServerCertificate=yes')
    #conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df = pd.concat(datalist)  

#df["Qty Balance"] = df.groupby(["company","itemcode"])["quantity"].cumsum()
#df["Amount Balance"] = df.groupby(["company","itemcode"])["amount"].cumsum()   
datax = df.pivot_table(index=["iday" ],columns=["Header","idivisi"],aggfunc=np.sum,values=["amount"],fill_value=0,margins=True ).sort_index().to_excel("/data/proyeksi.xlsx" )

 

#df.to_excel("/data/plgabungan.xlsx" )
#df.to_excel("/data/PIUTANG_2020x.xlsx",index=False)
#df.pivot_table(index=["Company", "Partner Name","Currency"],columns=["Aging"],aggfunc=np.sum,values=["BalanceIDR"],fill_value="0",margins=True ).to_excel(mpath + '/temp/'+ filename )


 

/tmp/ipykernel_422467/2117857880.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pandas.io.sql.read_sql(msg_sql,conn)
